In [79]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import wrds

# WRDS
try:
    db = wrds.Connection(wrds_username="wenxzeng")
except Exception as e:
    print(f"WRDS Connection Failed: {e}")
    exit(1)

# universe
TICKER_UNIVERSE = (
    'AAPL', 'MSFT', 'NVDA', 'AVGO', 'ADBE', 'UNH', 'JNJ', 'PFE', 'MRK', 'ABBV',
    'JPM', 'BAC', 'WFC', 'GS', 'MS', 'AMZN', 'TSLA', 'HD', 'MCD', 'NKE', 'GOOGL',
    'META', 'DIS', 'VZ', 'CMCSA', 'PG', 'KO', 'PEP', 'WMT', 'COST', 'XOM', 'CVX',
    'COP', 'BA', 'UNP', 'HON', 'NEE', 'DUK', 'SO', 'PLD', 'AMT', 'CCI', 'SHW', 'DOW'
)
startdate = '2017-01-01'
enddate = '2025-02-28'

# SQL
sql_request = f"""
SELECT 
    n.ticker,
    a.date,
    a.openprc AS open,
    a.askhi AS high,
    a.bidlo AS low,
    a.prc AS close,
    a.vol AS volume
FROM crsp.dsf a
JOIN crsp.msenames n
    ON a.permno = n.permno
WHERE n.ticker IN {TICKER_UNIVERSE}
AND a.date BETWEEN '{startdate}' AND '{enddate}'
ORDER BY n.ticker, a.date;
"""

print("Fetching data from WRDS...")
try:
    df = db.raw_sql(sql_request).drop_duplicates()
except Exception as e:
    print(f"SQL Query Failed: {e}")
    exit(1)


def calculate_technicals(df):
    
    df = df.sort_values(by='date').copy()

    # SMA & EMA）
    df['SMA_20'] = df['close'].rolling(20, min_periods=1).mean()
    df['EMA_12'] = df['close'].ewm(span=12, adjust=False).mean()

    #  RSI
    window = 14
    delta = df['close'].diff(1)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = pd.Series(gain).rolling(window=window, min_periods=window).mean()
    avg_loss = pd.Series(loss).rolling(window=window, min_periods=window).mean()
    avg_loss = avg_loss.replace(0, 1e-10)  # 避免除零错误

    rs = avg_gain / avg_loss
    df[f'RSI_{window}'] = 100 - (100 / (1 + rs.values))

    #  MACD
    df['EMA_26'] = df['close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']

    #  bollinger
    df['BB_Middle'] = df['close'].rolling(window=20, min_periods=1).mean()
    df['BB_Upper'] = df['BB_Middle'] + 2 * df['close'].rolling(window=20, min_periods=1).std()
    df['BB_Lower'] = df['BB_Middle'] - 2 * df['close'].rolling(window=20, min_periods=1).std()

    return df

print("Calculating technical indicators...")
df_technicals = df.groupby('ticker', group_keys=False).apply(calculate_technicals)
df_technicals_ = df_technicals.copy()
df_technicals_['date'] = pd.to_datetime(df_technicals_['date'])
df_technicals_['date'] = df_technicals_['date'].dt.strftime('%Y-%m-%d')
df_technicals_ = df_technicals_[df_technicals_.date >= '2018-01-01']
df_technicals_




Loading library list...
Done
Fetching data from WRDS...
Calculating technical indicators...


,ticker,date,open,high,low,close,volume,SMA_20,EMA_12,RSI_14,EMA_26,MACD,MACD_signal,MACD_hist,BB_Middle,BB_Upper,BB_Lower
1255,AAPL,2018-01-02,170.16000,172.30,169.25999,172.25999,25555934.0,171.951999,172.019347,48.882802,171.632371,0.386977,0.879065,-0.492089,171.951999,176.544129,167.359870
1260,AAPL,2018-01-03,172.53000,174.55,171.96001,172.23000,29517900.0,172.073499,172.051755,51.522121,171.676640,0.375116,0.778275,-0.403159,172.073499,176.553097,167.593902
1265,AAPL,2018-01-04,172.53999,173.47,172.08000,173.03000,22434597.0,172.242999,172.202255,52.154205,171.776888,0.425366,0.707693,-0.282327,172.242999,176.589460,167.896539
1270,AAPL,2018-01-05,173.44000,175.37,173.05000,175.00000,23660018.0,172.542500,172.632677,57.106369,172.015637,0.617039,0.689563,-0.072523,172.542500,176.774964,168.310036
1275,AAPL,2018-01-08,174.35001,175.61,173.92999,174.35001,20567766.0,172.794000,172.896882,51.029285,172.188554,0.708328,0.693316,0.015012,172.794000,176.812587,168.775413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112212,XOM,2024-12-24,106.52000,107.19,105.70000,106.40000,7806997.0,111.943500,108.866587,13.039400,112.219381,-3.352794,-2.797915,-0.554879,111.943500,121.011722,102.875278
112218,XOM,2024-12-26,106.52000,107.03,105.94000,106.49000,9653408.0,111.369500,108.500958,9.560976,111.794982,-3.294024,-2.897137,-0.396887,111.369500,120.283563,102.455437
112224,XOM,2024-12-27,106.30000,107.99,105.77000,106.48000,11943906.0,110.810500,108.190041,10.828729,111.401280,-3.211238,-2.959957,-0.251281,110.810500,119.461934,102.159066
112230,XOM,2024-12-30,106.30000,106.56,105.51000,105.76000,11080770.0,110.200500,107.816189,10.769231,110.983407,-3.167218,-3.001409,-0.165809,110.200500,118.440006,101.960994


In [77]:
df_technicals

,ticker,date,open,high,low,close,volume,SMA_20,EMA_12,RSI_14,EMA_26,MACD,MACD_signal,MACD_hist,BB_Middle,BB_Upper,BB_Lower
0,AAPL,2018-01-02,170.16000,172.30,169.25999,172.25999,25555934.0,172.259990,172.259990,NaN,172.259990,0.000000,0.000000,0.000000,172.259990,NaN,NaN
5,AAPL,2018-01-03,172.53000,174.55,171.96001,172.23000,29517900.0,172.244995,172.255376,NaN,172.257769,-0.002392,-0.000478,-0.001914,172.244995,172.287407,172.202583
10,AAPL,2018-01-04,172.53999,173.47,172.08000,173.03000,22434597.0,172.506663,172.374549,NaN,172.314971,0.059578,0.011533,0.048045,172.506663,173.413605,171.599722
15,AAPL,2018-01-05,173.44000,175.37,173.05000,175.00000,23660018.0,173.129998,172.778465,56.473658,172.513862,0.264603,0.062147,0.202456,173.129998,175.730976,170.529019
20,AAPL,2018-01-08,174.35001,175.61,173.92999,174.35001,20567766.0,173.374000,173.020241,33.186225,172.649873,0.370368,0.123791,0.246577,173.374000,175.876911,170.871089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41774,XOM,2024-12-24,106.52000,107.19,105.70000,106.40000,7806997.0,111.943500,108.866587,NaN,112.219381,-3.352794,-2.797915,-0.554879,111.943500,121.011722,102.875278
41780,XOM,2024-12-26,106.52000,107.03,105.94000,106.49000,9653408.0,111.369500,108.500958,NaN,111.794982,-3.294024,-2.897137,-0.396887,111.369500,120.283563,102.455437
41786,XOM,2024-12-27,106.30000,107.99,105.77000,106.48000,11943906.0,110.810500,108.190041,NaN,111.401280,-3.211238,-2.959957,-0.251281,110.810500,119.461934,102.159066
41792,XOM,2024-12-30,106.30000,106.56,105.51000,105.76000,11080770.0,110.200500,107.816189,NaN,110.983407,-3.167218,-3.001409,-0.165809,110.200500,118.440006,101.960994


In [80]:
# save
df_technicals_.to_csv('technical_data.csv', index=False)
print("Technical data saved to 'technical_data.csv' ")

Technical data saved to 'technical_data.csv' 
